<a href="https://colab.research.google.com/github/ParvezAlam-AI/AI-App-Challenge-2024/blob/main/WOKRING_code_after_testing_on_spaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working Code with simple chat bot.


## install module


In [4]:
!pip install gradio
!pip install yfinance
!pip install plotly
!pip install pandas
!pip install numpy
!pip install matplotlib

!pip insall requests
!pip install openai
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.4 MB/s eta 0:00:00
ERROR: unknown command "insall" - maybe you meant "

## import

In [5]:
from google.colab import userdata
import pandas as pd
import openai
import gradio as gr
import yfinance as yf
import json
import os
import matplotlib.pyplot as plt
import io
from PIL import Image
import plotly.graph_objs as go  # Import Plotly for interactive charts
import re
import datetime

openai.api_key = userdata.get('apopen')

## code

In [6]:

# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary")
    }
    return overview


# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "total_revenue": safe_get(info, "totalRevenue"),
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE")
    }
    return metrics



def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list




def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    ytd_return = ((current_price - history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]) / history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]) * 100
    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance





def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report






# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report






# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    }
]

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response






# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": assistant_response})
    # Update the history for Gradio display
    history.append((user_input, assistant_response))
    return history, history

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Stock Market Chatbot</center></h1>")
    chatbot = gr.Chatbot()
    state = gr.State([])  # To store the conversation history
    with gr.Row():
        txt = gr.Textbox(
            show_label=False,
            placeholder="Enter your message",
        )
    txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)  # Clear the input box after submission

# Launch the Gradio app
demo.launch()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ea0d029d003e6ffdd1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Testing new functions

## gradio code with stock chat and stock insight

In [7]:


# ... (your existing imports and code)

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": assistant_response})
    # Update the history for Gradio display
    history.append((user_input, assistant_response))
    return history, history

# Function to display stock insight
def display_stock_insight(ticker):
    # Fetch the data using your existing functions
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)
    # Build the layout as per your provided format
    # Format the data into a dictionary or string
    # For simplicity, we'll use Markdown strings here
    stock_insight_content = f"""
### Company Overview
**{overview.get('company_name')}**

{overview.get('description')}

**Name:** {overview.get('company_name')}
**CEO:** {overview.get('ceo')}
**Website:** [Visit Website](https://{overview.get('website')})
**Sector:** {overview.get('industry')}
**Year Founded:** {overview.get('year_founded')}

### Profile
**Market Cap:** {financials.get('market_cap')}
**Revenue:** {financials.get('total_revenue')}
**Employees:** {financials.get('employees')}

### Margins
**Gross Margin:** {financials.get('gross_profit_margin')}

### Stock Performance
**Current Price:** {performance.get('current_price')}
**52-Week Range:** {performance.get('52_week_range')}
**YTD Return:** {performance.get('ytd_return')}
"""
    return stock_insight_content

# Initialize the state for the stock ticker
stock_ticker = gr.State("V")  # Default ticker symbol

# Create Gradio interface with custom CSS
with gr.Blocks(css="""
/* Set the background image */
body {
    background-image: url('/file/app_bg.jpg');
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center center;
}
/* Optional: Adjust chat bubble styles for better visibility */
.gradio-chatbot {
    background-color: rgba(255, 255, 255, 0.8);
    border-radius: 10px;
}
/* Adjust text color for readability */
.gradio-chatbot .message.user {
    color: #000;
}
.gradio-chatbot .message.bot {
    color: #000;
}
""") as demo:
    gr.Markdown("<h1><center>Stock Market Assistant</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("## Stock Assistant Chat")
            chatbot = gr.Chatbot()
            chat_state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Enter your message",
                )
                txt.submit(chat, inputs=[txt, chat_state], outputs=[chatbot, chat_state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("## Stock Insight")
            stock_insight = gr.Markdown()
            # Update the stock insight based on the latest ticker from the chat
            def update_stock_insight(history):
                # Extract the last ticker symbol used in the conversation
                last_ticker = stock_ticker.value
                # Generate the stock insight content
                content = display_stock_insight(last_ticker)
                return content

            # Call update_stock_insight whenever the conversation updates
            chat_state.change(
                update_stock_insight,
                inputs=chat_state,
                outputs=stock_insight
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6db972f3544f723139.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## whole code test with new layout. This is working good. 06/10/2024 - 06:24 PM

In [8]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        # NEW CODE: Add website
        "website": safe_get(info, "website")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "enterprise_value": safe_get(info, "enterpriseValue"),  # NEW CODE
        "shares_outstanding": safe_get(info, "sharesOutstanding"),  # NEW CODE
        "total_revenue": safe_get(info, "totalRevenue"),
        "employees": safe_get(info, "fullTimeEmployees"),  # NEW CODE
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE"),
        # NEW CODE: Additional financial health metrics
        "cash": safe_get(info, "totalCash"),
        "net_debt": safe_get(info, "totalDebt") - safe_get(info, "totalCash") if info.get("totalDebt") and info.get("totalCash") else None,
        "debt_to_equity": safe_get(info, "debtToEquity"),
        "ebit_to_interest": safe_get(info, "ebitda") / safe_get(info, "interestExpense") if info.get("ebitda") and info.get("interestExpense") else None,
        # NEW CODE: Dividends
        "dividendYield": safe_get(info, "dividendYield"),
        "payoutRatio": safe_get(info, "payoutRatio"),
        "dividendRate": safe_get(info, "dividendRate"),
        # NEW CODE: Growth rates (placeholders)
        "revenue_3yr_cagr": None,
        "eps_diluted_3yr_cagr": None
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    }
]

# Initialize stock_ticker state
stock_ticker = {"value": "V"}  # NEW CODE: Initialize stock_ticker with default ticker symbol

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # NEW CODE: Update the stock_ticker state
        stock_ticker["value"] = ticker

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": assistant_response})
    # Update the history for Gradio display
    history.append((user_input, assistant_response))
    return history, history

# NEW CODE: Function to display stock insight
def display_stock_insight(ticker):
    # Fetch the data using your existing functions
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    # Format the data into sections
    stock_insight_content = f"""
### Company Overview


**Name:** {overview.get('company_name')}
**CEO:** {overview.get('ceo')}
**Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
**Sector:** {overview.get('industry')}
**Year Founded:** {overview.get('year_founded')}

### Profile
**Market Cap:** {financials.get('market_cap')}
**EV:** {financials.get('enterprise_value')}
**Shares Out:** {financials.get('shares_outstanding')}
**Revenue:** {financials.get('total_revenue')}
**Employees:** {financials.get('employees')}

### Margins
**Gross Margin:** {financials.get('gross_profit_margin')}
**EBITDA Margin:** {financials.get('ebitda_margin')}
**Operating Margin:** {financials.get('operating_margin')}
**Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
**Current Price:** {performance.get('current_price')}
**52-Week Range:** {performance.get('52_week_range')}
**YTD Return:** {performance.get('ytd_return')}
**1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
+
**Cash:** {financials.get('cash')}
**Net Debt:** {financials.get('net_debt')}
**Debt/Equity:** {financials.get('debt_to_equity')}
**EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
**Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
**EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
**Yield:** {financials.get('dividendYield')}
**Payout Ratio:** {financials.get('payoutRatio')}
**DPS:** {financials.get('dividendRate')}

{overview.get('description')}
"""
    return stock_insight_content

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS can be added here */
/* Adjust the overall layout and styling as needed */
""") as demo:
    gr.Markdown("<h1><center>Stock Market Assistant</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h1><center> Stock Assistant Chat</center></h1>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Enter your message",
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h1><center> Stock Insight</center></h1>")
            stock_insight = gr.Markdown()
            # Function to update stock insight
            def update_stock_insight(history):
                # Get the latest ticker symbol
                ticker = stock_ticker["value"]
                # Generate the stock insight content
                content = display_stock_insight(ticker)
                return content

            # Update stock insight whenever the conversation changes
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=stock_insight
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://367daa2244463050d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## with new layout and mordern look and clickable ticker and company name

In [22]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        # NEW CODE: Add website
        "website": safe_get(info, "website")
    }
    return overview

def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": format_number(safe_get(info, "marketCap")),
        "enterprise_value": format_number(safe_get(info, "enterpriseValue")),
        "shares_outstanding": format_number(safe_get(info, "sharesOutstanding")),
        "total_revenue": format_number(safe_get(info, "totalRevenue")),
        "employees": format_number(safe_get(info, "fullTimeEmployees")),
        "gross_profit_margin": f"{safe_get(info, 'grossMargins')*100:.2f}%" if safe_get(info, 'grossMargins') != "-" else "-",
        "ebitda_margin": f"{safe_get(info, 'ebitdaMargins')*100:.2f}%" if safe_get(info, 'ebitdaMargins') != "-" else "-",
        "operating_margin": f"{safe_get(info, 'operatingMargins')*100:.2f}%" if safe_get(info, 'operatingMargins') != "-" else "-",
        "net_profit_margin": f"{safe_get(info, 'profitMargins')*100:.2f}%" if safe_get(info, 'profitMargins') != "-" else "-",
        "eps_diluted": format_number(safe_get(info, "trailingEps")),
        "pe_ratio": format_number(safe_get(info, "trailingPE")),
        "forward_pe_ratio": format_number(safe_get(info, "forwardPE")),
        "cash": format_number(safe_get(info, "totalCash")),
        "net_debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash")) if safe_get(info, "totalDebt") != "-" and safe_get(info, "totalCash") != "-" else "N/A",
        "debt_to_equity": f"{safe_get(info, 'debtToEquity'):.2f}" if safe_get(info, "debtToEquity") != "-" else "-",
        "ebit_to_interest": f"{(safe_get(info, 'ebitda') / safe_get(info, 'interestExpense')):.2f}" if safe_get(info, "ebitda") != "-" and safe_get(info, "interestExpense") != "-" else "N/A",
        "dividendYield": f"{safe_get(info, 'dividendYield')*100:.2f}%" if safe_get(info, "dividendYield") != "-" else "-",
        "payoutRatio": f"{safe_get(info, 'payoutRatio')*100:.2f}%" if safe_get(info, "payoutRatio") != "-" else "-",
        "dividendRate": format_number(safe_get(info, "dividendRate")),
        "revenue_3yr_cagr": "N/A",
        "eps_diluted_3yr_cagr": "N/A"
    }
    return metrics


# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    }
]

# Initialize stock_ticker state
stock_ticker = {"value": "V"}  # NEW CODE: Initialize stock_ticker with default ticker symbol

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # NEW CODE: Update the stock_ticker state
        stock_ticker["value"] = ticker

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": assistant_response})
    # Update the history for Gradio display
    history.append((user_input, assistant_response))
    return history, history

# NEW CODE: Function to display stock insight
def display_stock_insight(ticker):
    # Fetch the data using your existing functions
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    # Format the data into sections
    stock_insight_content = f"""
### Company Overview


**Name:** {overview.get('company_name')}
**CEO:** {overview.get('ceo')}
**Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
**Sector:** {overview.get('industry')}
**Year Founded:** {overview.get('year_founded')}

### Profile
**Market Cap:** {financials.get('market_cap')}
**EV:** {financials.get('enterprise_value')}
**Shares Out:** {financials.get('shares_outstanding')}
**Revenue:** {financials.get('total_revenue')}
**Employees:** {financials.get('employees')}

### Margins
**Gross Margin:** {financials.get('gross_profit_margin')}
**EBITDA Margin:** {financials.get('ebitda_margin')}
**Operating Margin:** {financials.get('operating_margin')}
**Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
**Current Price:** {performance.get('current_price')}
**52-Week Range:** {performance.get('52_week_range')}
**YTD Return:** {performance.get('ytd_return')}
**1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
+
**Cash:** {financials.get('cash')}
**Net Debt:** {financials.get('net_debt')}
**Debt/Equity:** {financials.get('debt_to_equity')}
**EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
**Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
**EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
**Yield:** {financials.get('dividendYield')}
**Payout Ratio:** {financials.get('payoutRatio')}
**DPS:** {financials.get('dividendRate')}

{overview.get('description')}
"""
    return stock_insight_content

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS can be added here */
/* Adjust the overall layout and styling as needed */
""") as demo:
    gr.Markdown("<h1><center>Stock Market Assistant</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h1><center> Stock Assistant Chat</center></h1>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Enter your message",
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h1><center> Stock Insight</center></h1>")
            stock_insight = gr.Markdown()
            # Function to update stock insight
            def update_stock_insight(history):
                # Get the latest ticker symbol
                ticker = stock_ticker["value"]
                # Generate the stock insight content
                content = display_stock_insight(ticker)
                return content

            # Update stock insight whenever the conversation changes
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=stock_insight
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9b670183a502d60445.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [10]:
!pip install ace-tools

In [11]:
import pandas as pd
import yfinance as yf

# Safe get function to retrieve info from dictionaries, returning 0 if the value is not a number
def safe_get(data, key, default="-"):
    value = data.get(key) if data.get(key) is not None else default
    if isinstance(value, (int, float)):
        return value
    return default

# Function to format numbers to B (Billions), M (Millions), or with 2 decimal places
def format_number(value):
    if isinstance(value, (int, float)):
        if value >= 1e9:  # Billion
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:  # Million
            return f"{value / 1e6:.2f}M"
        else:
            return f"{value:.2f}"  # Up to 2 decimal places for smaller numbers
    return value  # Return value as is if it's not a number

# Function to get metrics from financial statements, ensuring they are numeric
def get_metric_from_statement(statement, metric, default="-"):
    try:
        value = statement.loc[metric].iloc[0] if metric in statement.index else default
        return value if isinstance(value, (int, float)) else default
    except (IndexError, KeyError):
        return default

# Helper function to safely divide two values
def safe_divide(numerator, denominator, default="-"):
    if isinstance(numerator, (int, float)) and isinstance(denominator, (int, float)) and denominator != 0:
        return numerator / denominator
    return default

# Function to get company overview and other financial data
def get_stock_information(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Use the balance sheet, income statement, and cash flow statements to calculate missing metrics
    balance_sheet = stock.balance_sheet
    income_statement = stock.financials
    cashflow_statement = stock.cashflow

    # Define the two columns
    data = {
        "Attribute": [
            "Company Overview", "Name", "CEO", "Website", "Sector", "Year Founded",
            "Market Cap", "EV", "Shares Out", "Revenue", "Employees",
            "Gross Margin", "EBITDA Margin", "Operating Margin", "Pre-Tax Margin", "Net Margin",
            "Free Cash Flow (FCF)", "ROA", "ROE", "ROCE", "ROIC",
            "P/E", "P/B", "EV/Sales", "EV/EBITDA", "Price Target (NTM)", "Cash", "Net Debt", "Debt/Equity"
        ],
        "Value": [
            safe_get(info, "longBusinessSummary"),
            safe_get(info, "longName"),
            safe_get(info, "ceo"),
            safe_get(info, "website"),
            safe_get(info, "industry"),
            safe_get(info, "startDate"),
            format_number(safe_get(info, "marketCap")),
            format_number(safe_get(info, "enterpriseValue")),
            format_number(safe_get(info, "sharesOutstanding")),
            format_number(safe_get(info, "totalRevenue")),
            format_number(safe_get(info, "fullTimeEmployees")),
            format_number(safe_get(info, "grossMargins")),
            format_number(safe_get(info, "ebitdaMargins")),
            format_number(safe_get(info, "operatingMargins")),
            format_number(safe_get(info, "preTaxProfitMargin")),
            format_number(safe_get(info, "profitMargins")),
            format_number(safe_get(info, "freeCashflow")),
            # ROA: Return on Assets, fallback to any available period
            format_number(safe_divide(get_metric_from_statement(income_statement, "Net Income"), get_metric_from_statement(balance_sheet, "Total Assets"))),
            # ROE: Return on Equity
            format_number(safe_divide(get_metric_from_statement(income_statement, "Net Income"), get_metric_from_statement(balance_sheet, "Total Stockholder Equity"))),
            # ROCE: Return on Capital Employed
            format_number(safe_divide(get_metric_from_statement(income_statement, "Net Income"), (get_metric_from_statement(balance_sheet, "Total Assets") - get_metric_from_statement(balance_sheet, "Current Liabilities")))),
            # ROIC: Return on Invested Capital
            format_number(safe_divide(get_metric_from_statement(income_statement, "Net Income"), (get_metric_from_statement(balance_sheet, "Total Stockholder Equity") + get_metric_from_statement(balance_sheet, "Total Liabilities")))),
            format_number(safe_get(info, "trailingPE")),
            format_number(safe_get(info, "priceToBook")),
            # EV/Sales
            format_number(safe_divide(get_metric_from_statement(balance_sheet, "Enterprise Value"), get_metric_from_statement(income_statement, "Total Revenue"))),
            # EV/EBITDA
            format_number(safe_divide(get_metric_from_statement(balance_sheet, "Enterprise Value"), get_metric_from_statement(income_statement, "EBITDA"))),
            # Price Target (NTM)
            format_number(safe_get(info, "targetMeanPrice")),
            format_number(safe_get(info, "totalCash")),
            # Net Debt (ensure both values are numerical before performing subtraction)
            format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash") if isinstance(safe_get(info, "totalDebt"), (int, float)) and isinstance(safe_get(info, "totalCash"), (int, float)) else "-"),
            format_number(safe_get(info, "debtToEquity"))
        ]
    }

    # Removing None rows (if needed)
    df = pd.DataFrame(data).replace("N/A", "-")  # Replace "N/A" with "-" in the DataFrame
    df = df.replace(0, "-")  # Replace zero values with "-" where applicable

    # Display the DataFrame (printing it here, but you can use other methods like saving to CSV)
    print(df)

    return df

# Example Usage
get_stock_information('TSLA')

               Attribute            Value
0       Company Overview                -
1                   Name                -
2                    CEO                -
3                Website                -
4                 Sector                -
5           Year Founded                -
6             Market Cap          798.92B
7                     EV          781.51B
8             Shares Out            3.19B
9                Revenue           95.32B
10             Employees        140473.00
11          Gross Margin             0.18
12         EBITDA Margin             0.13
13      Operating Margin             0.09
14        Pre-Tax Margin                -
15            Net Margin             0.13
16  Free Cash Flow (FCF)    -907249984.00
17                   ROA             0.14
18                   ROE                -
19                  ROCE             0.19
20                  ROIC                -
21                   P/E            70.45
22                   P/B          

,Attribute,Value
0,Company Overview,-
1,Name,-
2,CEO,-
3,Website,-
4,Sector,-
5,Year Founded,-
6,Market Cap,798.92B
7,EV,781.51B
8,Shares Out,3.19B
9,Revenue,95.32B


In [12]:
import pandas as pd
import yfinance as yf

# Function to safely get data from a dictionary, using "-" as the default value for missing data
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to format large numbers to billions (B) or millions (M) and limit decimals to two digits
def format_number(value):
    if isinstance(value, (int, float)):
        if value >= 1e9:  # Billion
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:  # Million
            return f"{value / 1e6:.2f}M"
        else:
            return f"{value:.2f}"
    return value  # Return as is if it's not a number

# Function to get the stock information
def get_company_report(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Company overview section
    company_overview = {
        "Name": safe_get(info, "longName"),
        "CEO": safe_get(info, "ceo"),
        "Website": safe_get(info, "website"),
        "Sector": safe_get(info, "sector"),
        "Year Founded": safe_get(info, "startDate")
    }

    # Profile section
    profile = {
        "Market Cap": format_number(safe_get(info, "marketCap")),
        "EV": format_number(safe_get(info, "enterpriseValue")),
        "Shares Out": format_number(safe_get(info, "sharesOutstanding")),
        "Revenue": format_number(safe_get(info, "totalRevenue")),
        "Employees": format_number(safe_get(info, "fullTimeEmployees"))
    }

    # Margins section
    margins = {
        "Gross": format_number(safe_get(info, "grossMargins") * 100) + "%" if safe_get(info, "grossMargins") != "-" else "-",
        "EBITDA": format_number(safe_get(info, "ebitdaMargins") * 100) + "%" if safe_get(info, "ebitdaMargins") != "-" else "-",
        "Operating": format_number(safe_get(info, "operatingMargins") * 100) + "%" if safe_get(info, "operatingMargins") != "-" else "-",
        "Pre-Tax": format_number(safe_get(info, "preTaxProfitMargin") * 100) + "%" if safe_get(info, "preTaxProfitMargin") != "-" else "-",
        "Net": format_number(safe_get(info, "profitMargins") * 100) + "%" if safe_get(info, "profitMargins") != "-" else "-"
    }

    # Returns (5Yr Avg) section
    returns = {
        "ROA": format_number(safe_get(info, "returnOnAssets") * 100) + "%" if safe_get(info, "returnOnAssets") != "-" else "-",
        "ROE": format_number(safe_get(info, "returnOnEquity") * 100) + "%" if safe_get(info, "returnOnEquity") != "-" else "-",
        "ROIC": format_number(safe_get(info, "returnOnInvestment") * 100) + "%" if safe_get(info, "returnOnInvestment") != "-" else "-"
    }

    # Valuation (TTM) section
    valuation_ttm = {
        "P/E": format_number(safe_get(info, "trailingPE")),
        "P/B": format_number(safe_get(info, "priceToBook")),
        "EV/Sales": format_number(safe_get(info, "enterpriseToRevenue")),
        "EV/EBITDA": format_number(safe_get(info, "enterpriseToEbitda")),
        "P/FCF": format_number(safe_get(info, "priceToFreeCashFlows")),
        "EV/Gross Profit": format_number(safe_get(info, "enterpriseToGrossProfit"))
    }

    # Valuation (NTM) section
    valuation_ntm = {
        "Price Target": format_number(safe_get(info, "targetMeanPrice")),
        "P/E": format_number(safe_get(info, "forwardPE")),
        "PEG": format_number(safe_get(info, "pegRatio")),
        "EV/Sales": format_number(safe_get(info, "enterpriseToRevenue")),
        "EV/EBITDA": format_number(safe_get(info, "enterpriseToEbitda"))
    }

    # Financial Health section
    ebitda = safe_get(info, "ebitda")
    interest_expense = safe_get(info, "interestExpense")

    # Make sure both values are numeric before dividing
    ebit_interest = format_number(ebitda / interest_expense) if isinstance(ebitda, (int, float)) and isinstance(interest_expense, (int, float)) and interest_expense != 0 else "-"

    financial_health = {
        "Cash": format_number(safe_get(info, "totalCash")),
        "Net Debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash") if safe_get(info, "totalDebt") and safe_get(info, "totalCash") else "-"),
        "Debt/Equity": format_number(safe_get(info, "debtToEquity")),
        "EBIT/Interest": ebit_interest
    }

    # Growth (CAGR) section
    growth_cagr = {
        "Rev 3Yr": format_number(safe_get(info, "threeYearGrowthRate") * 100) + "%" if safe_get(info, "threeYearGrowthRate") != "-" else "-",
        "Rev 5Yr": format_number(safe_get(info, "fiveYearAvgDividendYield") * 100) + "%" if safe_get(info, "fiveYearAvgDividendYield") != "-" else "-",
        "Rev 10Yr": "-",
        "Dil EPS 3Yr": "-",
        "EPS Fwd 2Yr": "-"
    }

    # Dividends section
    dividends = {
        "Yield": format_number(safe_get(info, "dividendYield") * 100) + "%" if safe_get(info, "dividendYield") != "-" else "-",
        "Payout": format_number(safe_get(info, "payoutRatio") * 100) + "%" if safe_get(info, "payoutRatio") != "-" else "-",
        "DPS": format_number(safe_get(info, "dividendRate")),
        "DPS Growth 3Yr": "-",
        "DPS Growth 5Yr": "-",
        "DPS Growth Fwd 2Yr": "-"
    }

    # Combine all sections into a single report
    report = {
        "Company Overview": company_overview,
        "Profile": profile,
        "Margins": margins,
        "Returns (5Yr Avg)": returns,
        "Valuation (TTM)": valuation_ttm,
        "Valuation (NTM)": valuation_ntm,
        "Financial Health": financial_health,
        "Growth (CAGR)": growth_cagr,
        "Dividends": dividends
    }

    return report

# Example usage
company_report = get_company_report("KO")  # Example using Coca-Cola's ticker symbol
df = pd.DataFrame(company_report)
print(df)


                                    Company Overview   Profile Margins  \
Name                           The Coca-Cola Company       NaN     NaN   
CEO                                                -       NaN     NaN   
Website             https://www.coca-colacompany.com       NaN     NaN   
Sector                            Consumer Defensive       NaN     NaN   
Year Founded                                       -       NaN     NaN   
Market Cap                                       NaN   302.42B     NaN   
EV                                               NaN   330.04B     NaN   
Shares Out                                       NaN     4.31B     NaN   
Revenue                                          NaN    46.46B     NaN   
Employees                                        NaN  79100.00     NaN   
Gross                                            NaN       NaN  60.53%   
EBITDA                                           NaN       NaN  32.08%   
Operating                             

In [13]:
import pandas as pd
import yfinance as yf

# Function to safely get data from a dictionary, using "-" as the default value for missing data
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to format large numbers to billions (B) or millions (M) and limit decimals to two digits
def format_number(value):
    if isinstance(value, (int, float)):
        if value >= 1e9:  # Billion
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:  # Million
            return f"{value / 1e6:.2f}M"
        else:
            return f"{value:.2f}"
    return value  # Return as is if it's not a number

# Function to get the stock information
def get_company_report(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Company overview section
    company_overview = {
        "Company Overview": {
            "Name": safe_get(info, "longName"),
            "CEO": safe_get(info, "ceo"),
            "Website": safe_get(info, "website"),
            "Sector": safe_get(info, "sector"),
            "Year Founded": safe_get(info, "startDate")
        }
    }

    # Profile section
    profile = {
        "Profile": {
            "Market Cap": format_number(safe_get(info, "marketCap")),
            "EV": format_number(safe_get(info, "enterpriseValue")),
            "Shares Out": format_number(safe_get(info, "sharesOutstanding")),
            "Revenue": format_number(safe_get(info, "totalRevenue")),
            "Employees": format_number(safe_get(info, "fullTimeEmployees"))
        }
    }

    # Margins section
    margins = {
        "Margins": {
            "Gross Margin": format_number(safe_get(info, "grossMargins") * 100) + "%" if safe_get(info, "grossMargins") != "-" else "-",
            "EBITDA Margin": format_number(safe_get(info, "ebitdaMargins") * 100) + "%" if safe_get(info, "ebitdaMargins") != "-" else "-",
            "Operating Margin": format_number(safe_get(info, "operatingMargins") * 100) + "%" if safe_get(info, "operatingMargins") != "-" else "-",
            "Pre-Tax Margin": format_number(safe_get(info, "preTaxProfitMargin") * 100) + "%" if safe_get(info, "preTaxProfitMargin") != "-" else "-",
            "Net Margin": format_number(safe_get(info, "profitMargins") * 100) + "%" if safe_get(info, "profitMargins") != "-" else "-"
        }
    }

    # Returns (5Yr Avg) section
    returns = {
        "Returns (5Yr Avg)": {
            "ROA": format_number(safe_get(info, "returnOnAssets") * 100) + "%" if safe_get(info, "returnOnAssets") != "-" else "-",
            "ROE": format_number(safe_get(info, "returnOnEquity") * 100) + "%" if safe_get(info, "returnOnEquity") != "-" else "-",
            "ROIC": format_number(safe_get(info, "returnOnInvestment") * 100) + "%" if safe_get(info, "returnOnInvestment") != "-" else "-"
        }
    }

    # Valuation (TTM) section
    valuation_ttm = {
        "Valuation (TTM)": {
            "P/E": format_number(safe_get(info, "trailingPE")),
            "P/B": format_number(safe_get(info, "priceToBook")),
            "EV/Sales": format_number(safe_get(info, "enterpriseToRevenue")),
            "EV/EBITDA": format_number(safe_get(info, "enterpriseToEbitda")),
            "P/FCF": format_number(safe_get(info, "priceToFreeCashFlows")),
            "EV/Gross Profit": format_number(safe_get(info, "enterpriseToGrossProfit"))
        }
    }

    # Valuation (NTM) section
    valuation_ntm = {
        "Valuation (NTM)": {
            "Price Target": format_number(safe_get(info, "targetMeanPrice")),
            "P/E": format_number(safe_get(info, "forwardPE")),
            "PEG": format_number(safe_get(info, "pegRatio")),
            "EV/Sales": format_number(safe_get(info, "enterpriseToRevenue")),
            "EV/EBITDA": format_number(safe_get(info, "enterpriseToEbitda"))
        }
    }

    # Financial Health section
    ebitda = safe_get(info, "ebitda")
    interest_expense = safe_get(info, "interestExpense")

    # Make sure both values are numeric before dividing
    ebit_interest = format_number(ebitda / interest_expense) if isinstance(ebitda, (int, float)) and isinstance(interest_expense, (int, float)) and interest_expense != 0 else "-"

    financial_health = {
        "Financial Health": {
            "Cash": format_number(safe_get(info, "totalCash")),
            "Net Debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash") if safe_get(info, "totalDebt") and safe_get(info, "totalCash") else "-"),
            "Debt/Equity": format_number(safe_get(info, "debtToEquity")),
            "EBIT/Interest": ebit_interest
        }
    }

    # Growth (CAGR) section
    growth_cagr = {
        "Growth (CAGR)": {
            "Rev 3Yr": format_number(safe_get(info, "threeYearGrowthRate") * 100) + "%" if safe_get(info, "threeYearGrowthRate") != "-" else "-",
            "Rev 5Yr": format_number(safe_get(info, "fiveYearAvgDividendYield") * 100) + "%" if safe_get(info, "fiveYearAvgDividendYield") != "-" else "-",
            "Rev 10Yr": "-",
            "Dil EPS 3Yr": "-",
            "EPS Fwd 2Yr": "-"
        }
    }

    # Dividends section
    dividends = {
        "Dividends": {
            "Yield": format_number(safe_get(info, "dividendYield") * 100) + "%" if safe_get(info, "dividendYield") != "-" else "-",
            "Payout": format_number(safe_get(info, "payoutRatio") * 100) + "%" if safe_get(info, "payoutRatio") != "-" else "-",
            "DPS": format_number(safe_get(info, "dividendRate")),
            "DPS Growth 3Yr": "-",
            "DPS Growth 5Yr": "-",
            "DPS Growth Fwd 2Yr": "-"
        }
    }

    # Combine all sections into a single dictionary
    full_report = {**company_overview, **profile, **margins, **returns, **valuation_ttm, **valuation_ntm, **financial_health, **growth_cagr, **dividends}

    return pd.DataFrame(full_report)

# Example usage
company_report_df = get_company_report("KO")  # Example using Coca-Cola's ticker symbol
print(company_report_df)


                                    Company Overview   Profile Margins  \
Name                           The Coca-Cola Company       NaN     NaN   
CEO                                                -       NaN     NaN   
Website             https://www.coca-colacompany.com       NaN     NaN   
Sector                            Consumer Defensive       NaN     NaN   
Year Founded                                       -       NaN     NaN   
Market Cap                                       NaN   302.42B     NaN   
EV                                               NaN   330.04B     NaN   
Shares Out                                       NaN     4.31B     NaN   
Revenue                                          NaN    46.46B     NaN   
Employees                                        NaN  79100.00     NaN   
Gross Margin                                     NaN       NaN  60.53%   
EBITDA Margin                                    NaN       NaN  32.08%   
Operating Margin                      

In [14]:


# Safe get function to retrieve info from dictionaries, use blank if not available
def safe_get(data, key, default=""):
    return data.get(key, default) if data.get(key) is not None else default

# Function to format large numbers (B for Billions, M for Millions)
def format_number(value):
    if isinstance(value, (int, float)):
        if value >= 1e9:  # Billions
            return f"{value / 1e9:.2f}B"
        elif value >= 1e6:  # Millions
            return f"{value / 1e6:.2f}M"
        return f"{value:.2f}"  # Up to 2 decimal places
    return value

# Function to fetch all required data using yfinance
def get_company_report(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info  # Directly using the 'info' attribute to fetch company data

    # Build the report as a dictionary of key-value pairs
    report = {
        "Name": safe_get(info, "longName"),
        "CEO": safe_get(info, "ceo"),
        "Website": safe_get(info, "website"),
        "Sector": safe_get(info, "sector"),
        "Year Founded": safe_get(info, "startDate"),
        "Market Cap": format_number(safe_get(info, "marketCap")),
        "EV": format_number(safe_get(info, "enterpriseValue")),
        "Shares Outstanding": format_number(safe_get(info, "sharesOutstanding")),
        "Revenue": format_number(safe_get(info, "totalRevenue")),
        "Employees": format_number(safe_get(info, "fullTimeEmployees")),
        "Gross Margin": format_number(safe_get(info, "grossMargins") * 100) + "%" if safe_get(info, "grossMargins") != "" else "",
        "EBITDA Margin": format_number(safe_get(info, "ebitdaMargins") * 100) + "%" if safe_get(info, "ebitdaMargins") != "" else "",
        "Operating Margin": format_number(safe_get(info, "operatingMargins") * 100) + "%" if safe_get(info, "operatingMargins") != "" else "",
        "Net Margin": format_number(safe_get(info, "profitMargins") * 100) + "%" if safe_get(info, "profitMargins") != "" else "",
        "ROA": format_number(safe_get(info, "returnOnAssets") * 100) + "%" if safe_get(info, "returnOnAssets") != "" else "",
        "ROE": format_number(safe_get(info, "returnOnEquity") * 100) + "%" if safe_get(info, "returnOnEquity") != "" else "",
        "ROIC": format_number(safe_get(info, "returnOnInvestment") * 100) + "%" if safe_get(info, "returnOnInvestment") != "" else "",
        "P/E": format_number(safe_get(info, "trailingPE")),
        "P/B": format_number(safe_get(info, "priceToBook")),
        "EV/Sales": format_number(safe_get(info, "enterpriseToRevenue")),
        "EV/EBITDA": format_number(safe_get(info, "enterpriseToEbitda")),
        "P/FCF": format_number(safe_get(info, "priceToFreeCashFlows")),
        "EV/Gross Profit": format_number(safe_get(info, "enterpriseToGrossProfit")),
        "Cash": format_number(safe_get(info, "totalCash")),
        "Net Debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash") if safe_get(info, "totalDebt") and safe_get(info, "totalCash") else ""),
        "Debt/Equity": format_number(safe_get(info, "debtToEquity")),
        "Price Target": format_number(safe_get(info, "targetMeanPrice")),
        "Dividend Yield": format_number(safe_get(info, "dividendYield") * 100) + "%" if safe_get(info, "dividendYield") != "" else "",
        "Payout Ratio": format_number(safe_get(info, "payoutRatio") * 100) + "%" if safe_get(info, "payoutRatio") != "" else "",
        "Dividend Rate": format_number(safe_get(info, "dividendRate"))
    }

    # Convert the report to a DataFrame for display
    df = pd.DataFrame(report.items(), columns=["Metric", "Value"])
    return df

# Example usage
company_report_df = get_company_report("KO")  # Example using Coca-Cola ticker
print(company_report_df)


                Metric                             Value
0                 Name             The Coca-Cola Company
1                  CEO                                  
2              Website  https://www.coca-colacompany.com
3               Sector                Consumer Defensive
4         Year Founded                                  
5           Market Cap                           302.42B
6                   EV                           330.04B
7   Shares Outstanding                             4.31B
8              Revenue                            46.46B
9            Employees                          79100.00
10        Gross Margin                            60.53%
11       EBITDA Margin                            32.08%
12    Operating Margin                            32.45%
13          Net Margin                            22.92%
14                 ROA                             8.65%
15                 ROE                            38.77%
16                ROIC         

## code with formatted numbers


In [15]:
import yfinance as yf
import openai
import json
import re
import pandas as pd
import plotly.graph_objs as go
import gradio as gr

# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Helper function to format numbers in B/M/K with up to two decimal places
def format_number(num):
    if num is None:
        return "N/A"
    try:
        num = float(num)
        if num >= 1_000_000_000:
            return f"{num / 1_000_000_000:.2f}B"
        elif num >= 1_000_000:
            return f"{num / 1_000_000:.2f}M"
        elif num >= 1_000:
            return f"{num / 1_000:.2f}K"
        else:
            return f"{num:.2f}"
    except:
        return "N/A"

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        # NEW CODE: Add website
        "website": safe_get(info, "website")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": format_number(safe_get(info, "marketCap")),
        "enterprise_value": format_number(safe_get(info, "enterpriseValue")),  # NEW CODE
        "shares_outstanding": format_number(safe_get(info, "sharesOutstanding")),  # NEW CODE
        "total_revenue": format_number(safe_get(info, "totalRevenue")),
        "employees": format_number(safe_get(info, "fullTimeEmployees")),  # NEW CODE
        "gross_profit_margin": f"{safe_get(info, 'grossMargins')*100:.2f}%" if safe_get(info, 'grossMargins') != "-" else "-",
        "ebitda_margin": f"{safe_get(info, 'ebitdaMargins')*100:.2f}%" if safe_get(info, 'ebitdaMargins') != "-" else "-",
        "operating_margin": f"{safe_get(info, 'operatingMargins')*100:.2f}%" if safe_get(info, 'operatingMargins') != "-" else "-",
        "net_profit_margin": f"{safe_get(info, 'profitMargins')*100:.2f}%" if safe_get(info, 'profitMargins') != "-" else "-",
        "eps_diluted": format_number(safe_get(info, "trailingEps")),
        "pe_ratio": format_number(safe_get(info, "trailingPE")),
        "forward_pe_ratio": format_number(safe_get(info, "forwardPE")),
        # NEW CODE: Additional financial health metrics
        "cash": format_number(safe_get(info, "totalCash")),
        "net_debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash")) if safe_get(info, "totalDebt") != "-" and safe_get(info, "totalCash") != "-" else "N/A",
        "debt_to_equity": f"{safe_get(info, 'debtToEquity'):.2f}" if safe_get(info, "debtToEquity") != "-" else "-",
        "ebit_to_interest": f"{(safe_get(info, 'ebitda') / safe_get(info, 'interestExpense')):.2f}" if safe_get(info, "ebitda") != "-" and safe_get(info, "interestExpense") != "-" else "N/A",
        # NEW CODE: Dividends
        "dividendYield": f"{safe_get(info, 'dividendYield')*100:.2f}%" if safe_get(info, "dividendYield") != "-" else "-",
        "payoutRatio": f"{safe_get(info, 'payoutRatio')*100:.2f}%" if safe_get(info, "payoutRatio") != "-" else "-",
        "dividendRate": format_number(safe_get(info, "dividendRate")),
        # NEW CODE: Growth rates (placeholders)
        "revenue_3yr_cagr": "N/A",  # Placeholder as actual calculation requires historical data
        "eps_diluted_3yr_cagr": "N/A"  # Placeholder as actual calculation requires historical data
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        published_time = pd.to_datetime(item.get("providerPublishTime"), unit='s').strftime('%Y-%m-%d') if item.get("providerPublishTime") else "N/A"
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": published_time
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": format_number(current_price),
        "52_week_range": f"{format_number(fifty_two_week_low)} - {format_number(fifty_two_week_high)}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to create an interactive stock chart using Plotly
def create_stock_chart(ticker):
    stock = yf.Ticker(ticker)
    history = stock.history(period="10y")
    if history.empty:
        # Handle case where history is not available
        fig = go.Figure()
        fig.update_layout(
            title=f"No historical data available for {ticker}",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_dark"
        )
        return fig

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=history.index,
            y=history['Close'],
            mode='lines',
            name='Close Price',
            line=dict(color='purple'),
            fill='tozeroy',
            fillcolor='rgba(128, 0, 128, 0.2)'  # Semi-transparent purple
        )
    )
    fig.update_layout(
        title=f"Stock Price History for {ticker}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_dark",
        hovermode="x unified"
    )
    return fig

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Enterprise Value:** {financials.get('enterprise_value')}
- **Shares Outstanding:** {financials.get('shares_outstanding')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

**Dividends**
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    }
]

# Initialize stock_ticker state
stock_ticker = {"value": "V"}  # NEW CODE: Initialize stock_ticker with default ticker symbol

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # NEW CODE: Update the stock_ticker state
        stock_ticker["value"] = ticker

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Enterprise Value:** {metrics.get('enterprise_value')}
- **Shares Outstanding:** {metrics.get('shares_outstanding')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Employees:** {metrics.get('employees')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EBITDA Margin:** {metrics.get('ebitda_margin')}
- **Operating Margin:** {metrics.get('operating_margin')}
- **Net Profit Margin:** {metrics.get('net_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
- **Forward P/E Ratio:** {metrics.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {metrics.get('cash')}
- **Net Debt:** {metrics.get('net_debt')}
- **Debt/Equity:** {metrics.get('debt_to_equity')}
- **EBIT/Interest:** {metrics.get('ebit_to_interest')}

**Dividends**
- **Yield:** {metrics.get('dividendYield')}
- **Payout Ratio:** {metrics.get('payoutRatio')}
- **DPS:** {metrics.get('dividendRate')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
- **5-Year Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10-Year Total Return CAGR:** {performance.get('10y_total_return_cagr')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# Function to display stock insight
def display_stock_insight(ticker):
    # Fetch the data using existing functions
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    # Format the data into sections
    stock_insight_content = f"""
### Company Overview
- **Name:** {overview.get('company_name')}
- **CEO:** {overview.get('ceo')}
- **Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
- **Sector:** {overview.get('industry')}
- **Year Founded:** {overview.get('year_founded')}

### Profile
- **Market Cap:** {financials.get('market_cap')}
- **EV:** {financials.get('enterprise_value')}
- **Shares Out:** {financials.get('shares_outstanding')}
- **Revenue:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}

### Margins
- **Gross Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
- **Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
- **EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Description:** {overview.get('description')}
"""
    return stock_insight_content

# Function to create an interactive stock chart
def update_stock_insight(history):
    # Get the latest ticker symbol
    ticker = stock_ticker["value"]
    # Generate the stock insight content
    content = display_stock_insight(ticker)
    # Create the stock chart
    fig = create_stock_chart(ticker)
    return fig, content

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    history.append((user_input, assistant_response))
    return history, history

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS can be added here */
/* Adjust the overall layout and styling as needed */
""") as demo:
    gr.Markdown("<h1><center>Stock Market Assistant</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h2><center>Stock Assistant Chat</center></h2>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Enter your message",
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h2><center>Stock Insight</center></h2>")
            with gr.Column():
                stock_chart = gr.Plot()
                stock_insight = gr.Markdown()
            # Function to update stock insight
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=[stock_chart, stock_insight]
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a75dc46e959f133aed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [16]:
import yfinance as yf
import openai
import json
import re
import pandas as pd
import plotly.graph_objs as go
import gradio as gr

# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Helper function to format numbers in B/M/K with up to two decimal places
def format_number(num):
    if num is None:
        return "N/A"
    try:
        num = float(num)
        if num >= 1_000_000_000:
            return f"{num / 1_000_000_000:.2f}B"
        elif num >= 1_000_000:
            return f"{num / 1_000_000:.2f}M"
        elif num >= 1_000:
            return f"{num / 1_000:.2f}K"
        else:
            return f"{num:.2f}"
    except:
        return "N/A"

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        # NEW CODE: Add website
        "website": safe_get(info, "website")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": format_number(safe_get(info, "marketCap")),
        "enterprise_value": format_number(safe_get(info, "enterpriseValue")),  # NEW CODE
        "shares_outstanding": format_number(safe_get(info, "sharesOutstanding")),  # NEW CODE
        "total_revenue": format_number(safe_get(info, "totalRevenue")),
        "employees": format_number(safe_get(info, "fullTimeEmployees")),  # NEW CODE
        "gross_profit_margin": f"{safe_get(info, 'grossMargins')*100:.2f}%" if safe_get(info, 'grossMargins') != "-" else "-",
        "ebitda_margin": f"{safe_get(info, 'ebitdaMargins')*100:.2f}%" if safe_get(info, 'ebitdaMargins') != "-" else "-",
        "operating_margin": f"{safe_get(info, 'operatingMargins')*100:.2f}%" if safe_get(info, 'operatingMargins') != "-" else "-",
        "net_profit_margin": f"{safe_get(info, 'profitMargins')*100:.2f}%" if safe_get(info, 'profitMargins') != "-" else "-",
        "eps_diluted": format_number(safe_get(info, "trailingEps")),
        "pe_ratio": format_number(safe_get(info, "trailingPE")),
        "forward_pe_ratio": format_number(safe_get(info, "forwardPE")),
        # NEW CODE: Additional financial health metrics
        "cash": format_number(safe_get(info, "totalCash")),
        "net_debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash")) if safe_get(info, "totalDebt") != "-" and safe_get(info, "totalCash") != "-" else "N/A",
        "debt_to_equity": f"{safe_get(info, 'debtToEquity'):.2f}" if safe_get(info, "debtToEquity") != "-" else "-",
        "ebit_to_interest": f"{(safe_get(info, 'ebitda') / safe_get(info, 'interestExpense')):.2f}" if safe_get(info, "ebitda") != "-" and safe_get(info, "interestExpense") != "-" else "N/A",
        # NEW CODE: Dividends
        "dividendYield": f"{safe_get(info, 'dividendYield')*100:.2f}%" if safe_get(info, "dividendYield") != "-" else "-",
        "payoutRatio": f"{safe_get(info, 'payoutRatio')*100:.2f}%" if safe_get(info, "payoutRatio") != "-" else "-",
        "dividendRate": format_number(safe_get(info, "dividendRate")),
        # NEW CODE: Growth rates (placeholders)
        "revenue_3yr_cagr": "N/A",  # Placeholder as actual calculation requires historical data
        "eps_diluted_3yr_cagr": "N/A"  # Placeholder as actual calculation requires historical data
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        published_time = pd.to_datetime(item.get("providerPublishTime"), unit='s').strftime('%Y-%m-%d') if item.get("providerPublishTime") else "N/A"
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": published_time
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": format_number(current_price),
        "52_week_range": f"{format_number(fifty_two_week_low)} - {format_number(fifty_two_week_high)}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to create an interactive stock chart using Plotly
def create_stock_chart(ticker):
    stock = yf.Ticker(ticker)
    history = stock.history(period="10y")
    if history.empty:
        # Handle case where history is not available
        fig = go.Figure()
        fig.update_layout(
            title=f"No historical data available for {ticker}",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        return fig

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=history.index,
            y=history['Close'],
            mode='lines',
            name='Close Price',
            line=dict(color='purple'),
            fill='tozeroy',
            fillcolor='rgba(128, 0, 128, 0.2)'  # Semi-transparent purple
        )
    )
    fig.update_layout(
        title=f"Stock Price History for {ticker}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",  # Changed to 'plotly_white' for a cleaner look
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        )
    )
    return fig

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Enterprise Value:** {financials.get('enterprise_value')}
- **Shares Outstanding:** {financials.get('shares_outstanding')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

**Dividends**
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {"type": "string", "description": "The stock ticker symbol."}
            },
            "required": ["ticker"]
        }
    }
]

# Initialize stock_ticker state
stock_ticker = {"value": "V"}  # NEW CODE: Initialize stock_ticker with default ticker symbol

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # NEW CODE: Update the stock_ticker state
        stock_ticker["value"] = ticker

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Enterprise Value:** {metrics.get('enterprise_value')}
- **Shares Outstanding:** {metrics.get('shares_outstanding')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Employees:** {metrics.get('employees')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EBITDA Margin:** {metrics.get('ebitda_margin')}
- **Operating Margin:** {metrics.get('operating_margin')}
- **Net Profit Margin:** {metrics.get('net_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
- **Forward P/E Ratio:** {metrics.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {metrics.get('cash')}
- **Net Debt:** {metrics.get('net_debt')}
- **Debt/Equity:** {metrics.get('debt_to_equity')}
- **EBIT/Interest:** {metrics.get('ebit_to_interest')}

**Dividends**
- **Yield:** {metrics.get('dividendYield')}
- **Payout Ratio:** {metrics.get('payoutRatio')}
- **DPS:** {metrics.get('dividendRate')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
- **5-Year Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10-Year Total Return CAGR:** {performance.get('10y_total_return_cagr')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# Function to display stock insight
def display_stock_insight(ticker):
    # Fetch the data using existing functions
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    # Format the data into sections
    stock_insight_content = f"""
### Company Overview
- **Name:** {overview.get('company_name')}
- **CEO:** {overview.get('ceo')}
- **Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
- **Sector:** {overview.get('industry')}
- **Year Founded:** {overview.get('year_founded')}

### Profile
- **Market Cap:** {financials.get('market_cap')}
- **EV:** {financials.get('enterprise_value')}
- **Shares Out:** {financials.get('shares_outstanding')}
- **Revenue:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}

### Margins
- **Gross Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
- **Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
- **EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Description:** {overview.get('description')}
"""
    return stock_insight_content

# Function to create an interactive stock chart
def update_stock_insight(history):
    # Get the latest ticker symbol
    ticker = stock_ticker["value"]
    # Generate the stock insight content
    content = display_stock_insight(ticker)
    # Create the stock chart
    fig = create_stock_chart(ticker)
    return fig, content

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []
    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})
    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation)
    # Append the assistant's response to the conversation
    history.append((user_input, assistant_response))
    return history, history

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS for White Chat Background and Enhanced Appearance */

/* Style the entire Gradio container */
.gradio-container {
    background-color: #ffffff; /* White background for the entire application */
}

/* Style individual chat messages */
.chatbot .message {
    background-color: #ffffff; /* White background for messages */
    border-radius: 10px;
    padding: 10px;
    margin-bottom: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}

/* Differentiate user and assistant messages with border accents */
.chatbot .message.user {
    border-left: 4px solid #6c757d; /* Grey accent for user messages */
}

.chatbot .message.assistant {
    border-left: 4px solid #6c757d; /* Grey accent for assistant messages */
}

/* Optional: Style the chat scrollbar */
.chatbot .scroll-container::-webkit-scrollbar {
    width: 8px;
}

.chatbot .scroll-container::-webkit-scrollbar-track {
    background: #f1f1f1;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb {
    background: #888;
    border-radius: 4px;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb:hover {
    background: #555;
}

/* Style the Stock Insight section */
#stock_insight_section {
    background-color: #ffffff; /* White background */
    padding: 15px;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}
""") as demo:
    gr.Markdown("<h1><center>Stock Market Assistant</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h2><center>Stock Assistant Chat</center></h2>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Enter your message",
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h2><center>Stock Insight</center></h2>")
            with gr.Column(elem_id="stock_insight_section"):
                stock_chart = gr.Plot()
                stock_insight = gr.Markdown()
            # Function to update stock insight
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=[stock_chart, stock_insight]
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://05d4443534e8ab3b90.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [19]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Helper function to format numbers in B/M/K with up to two decimal places
def format_number(num):
    if num is None:
        return "N/A"
    try:
        num = float(num)
        if num >= 1_000_000_000:
            return f"{num / 1_000_000_000:.2f}B"
        elif num >= 1_000_000:
            return f"{num / 1_000_000:.2f}M"
        elif num >= 1_000:
            return f"{num / 1_000:.2f}K"
        else:
            return f"{num:.2f}"
    except:
        return "N/A"

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        "website": safe_get(info, "website")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": format_number(safe_get(info, "marketCap")),
        "enterprise_value": format_number(safe_get(info, "enterpriseValue")),
        "shares_outstanding": format_number(safe_get(info, "sharesOutstanding")),
        "total_revenue": format_number(safe_get(info, "totalRevenue")),
        "employees": format_number(safe_get(info, "fullTimeEmployees")),
        "gross_profit_margin": f"{safe_get(info, 'grossMargins')*100:.2f}%" if safe_get(info, 'grossMargins') != "-" else "-",
        "ebitda_margin": f"{safe_get(info, 'ebitdaMargins')*100:.2f}%" if safe_get(info, 'ebitdaMargins') != "-" else "-",
        "operating_margin": f"{safe_get(info, 'operatingMargins')*100:.2f}%" if safe_get(info, 'operatingMargins') != "-" else "-",
        "net_profit_margin": f"{safe_get(info, 'profitMargins')*100:.2f}%" if safe_get(info, 'profitMargins') != "-" else "-",
        "eps_diluted": format_number(safe_get(info, "trailingEps")),
        "pe_ratio": format_number(safe_get(info, "trailingPE")),
        "forward_pe_ratio": format_number(safe_get(info, "forwardPE")),
        "cash": format_number(safe_get(info, "totalCash")),
        "net_debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash")) if safe_get(info, "totalDebt") != "-" and safe_get(info, "totalCash") != "-" else "N/A",
        "debt_to_equity": f"{safe_get(info, 'debtToEquity'):.2f}" if safe_get(info, "debtToEquity") != "-" else "-",
        "ebit_to_interest": f"{(safe_get(info, 'ebitda') / safe_get(info, 'interestExpense')):.2f}" if safe_get(info, "ebitda") != "-" and safe_get(info, "interestExpense") != "-" else "N/A",
        "dividendYield": f"{safe_get(info, 'dividendYield')*100:.2f}%" if safe_get(info, "dividendYield") != "-" else "-",
        "payoutRatio": f"{safe_get(info, 'payoutRatio')*100:.2f}%" if safe_get(info, "payoutRatio") != "-" else "-",
        "dividendRate": format_number(safe_get(info, "dividendRate")),
        "revenue_3yr_cagr": "N/A",
        "eps_diluted_3yr_cagr": "N/A"
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        published_time = pd.to_datetime(item.get("providerPublishTime"), unit='s').strftime('%Y-%m-%d') if item.get("providerPublishTime") else "N/A"
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": published_time
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": format_number(current_price),
        "52_week_range": f"{format_number(fifty_two_week_low)} - {format_number(fifty_two_week_high)}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to create an interactive stock chart using Plotly
def create_stock_chart(ticker):
    stock = yf.Ticker(ticker)
    history = stock.history(period="10y")
    if history.empty:
        fig = go.Figure()
        fig.update_layout(
            title=f"No historical data available for {ticker}",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        return fig

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=history.index,
            y=history['Close'],
            mode='lines',
            name='Close Price',
            line=dict(color='purple'),
            fill='tozeroy',
            fillcolor='rgba(128, 0, 128, 0.2)'  # Semi-transparent purple
        )
    )
    fig.update_layout(
        title=f"Stock Price History for {ticker}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        )
    )
    return fig

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Enterprise Value:** {financials.get('enterprise_value')}
- **Shares Outstanding:** {financials.get('shares_outstanding')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

**Dividends**
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to update stock insight
def update_stock_insight(history):
    # Get the latest ticker symbol or fallback to the last discussed stock
    ticker = stock_ticker["value"]

    # If no conversation exists or no new stock ticker, use the last discussed ticker
    if not history or not ticker:
        ticker = stock_ticker["value"]  # Fallback to the last discussed stock

    # Generate the stock insight content
    content = display_stock_insight(ticker)

    # Create the stock chart
    fig = create_stock_chart(ticker)

    return fig, content

# Gradio Interface Function
def chat(user_input, history):
    if history is None:
        history = []

    # Build the conversation for stock_chat
    conversation = []
    for user_msg, assistant_msg in history:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": assistant_msg})

    # Append the new user input to the conversation
    conversation.append({"role": "user", "content": user_input})

    # Get assistant response
    assistant_response = stock_chat(conversation)

    # Append the assistant's response to the conversation
    history.append((user_input, assistant_response))

    # Ensure the last discussed stock ticker is stored for stock insight
    if stock_ticker["value"]:
        stock_ticker["value"] = get_ticker_from_conversation(conversation) or stock_ticker["value"]

    return history, history

# Function to display stock insight
def display_stock_insight(ticker):
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    stock_insight_content = f"""
### Company Overview
- **Name:** {overview.get('company_name')}
- **CEO:** {overview.get('ceo')}
- **Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
- **Sector:** {overview.get('industry')}
- **Year Founded:** {overview.get('year_founded')}

### Profile
- **Market Cap:** {financials.get('market_cap')}
- **EV:** {financials.get('enterprise_value')}
- **Shares Out:** {financials.get('shares_outstanding')}
- **Revenue:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}

### Margins
- **Gross Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
- **Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
- **EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Description:** {overview.get('description')}
"""
    return stock_insight_content

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS for White Chat Background and Enhanced Appearance */

/* Style the entire Gradio container */
.gradio-container {
    background-color: #ffffff; /* White background for the entire application */
}

/* Style individual chat messages */
.chatbot .message {
    background-color: #ffffff; /* White background for messages */
    border-radius: 10px;
    padding: 10px;
    margin-bottom: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}

/* Differentiate user and assistant messages with border accents */
.chatbot .message.user {
    border-left: 4px solid #6c757d; /* Grey accent for user messages */
}

.chatbot .message.assistant {
    border-left: 4px solid #6c757d; /* Grey accent for assistant messages */
}

/* Optional: Style the chat scrollbar */
.chatbot .scroll-container::-webkit-scrollbar {
    width: 8px;
}

.chatbot .scroll-container::-webkit-scrollbar-track {
    background: #f1f1f1;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb {
    background: #888;
    border-radius: 4px;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb:hover {
    background: #555;
}

/* Style the Stock Insight section */
#stock_insight_section {
    background-color: #ffffff; /* White background */
    padding: 15px;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}

/* Animation for the textbox placeholder */
@keyframes placeholderMove {
    0% {
        opacity: 0;
        transform: translateX(0);
    }
    50% {
        opacity: 1;
        transform: translateX(10px);
    }
    100% {
        opacity: 0;
        transform: translateX(-10px);
    }
}

.animated-placeholder::placeholder {
    animation: placeholderMove 4s infinite;
    color: #6c757d; /* Grey color for placeholder text */
}
""") as demo:
    gr.Markdown("<h1><center>Talk 2 Stock</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h2><center>Chat Xchange</center></h2>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Hey, Trader. Ask me -> How is Tesla doing today?",
                    elem_classes="animated-placeholder"
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h2><center>Stock Insight</center></h2>")
            with gr.Column(elem_id="stock_insight_section"):
                stock_chart = gr.Plot()
                stock_insight = gr.Markdown()
            # Function to update stock insight
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=[stock_chart, stock_insight]
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ad545d3495f509ebda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [23]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Helper function to format numbers in B/M/K with up to two decimal places
def format_number(num):
    if num is None:
        return "N/A"
    try:
        num = float(num)
        if num >= 1_000_000_000:
            return f"{num / 1_000_000_000:.2f}B"
        elif num >= 1_000_000:
            return f"{num / 1_000_000:.2f}M"
        elif num >= 1_000:
            return f"{num / 1_000:.2f}K"
        else:
            return f"{num:.2f}"
    except:
        return "N/A"

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary"),
        "website": safe_get(info, "website")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": format_number(safe_get(info, "marketCap")),
        "enterprise_value": format_number(safe_get(info, "enterpriseValue")),
        "shares_outstanding": format_number(safe_get(info, "sharesOutstanding")),
        "total_revenue": format_number(safe_get(info, "totalRevenue")),
        "employees": format_number(safe_get(info, "fullTimeEmployees")),
        "gross_profit_margin": f"{safe_get(info, 'grossMargins')*100:.2f}%" if safe_get(info, 'grossMargins') != "-" else "-",
        "ebitda_margin": f"{safe_get(info, 'ebitdaMargins')*100:.2f}%" if safe_get(info, 'ebitdaMargins') != "-" else "-",
        "operating_margin": f"{safe_get(info, 'operatingMargins')*100:.2f}%" if safe_get(info, 'operatingMargins') != "-" else "-",
        "net_profit_margin": f"{safe_get(info, 'profitMargins')*100:.2f}%" if safe_get(info, 'profitMargins') != "-" else "-",
        "eps_diluted": format_number(safe_get(info, "trailingEps")),
        "pe_ratio": format_number(safe_get(info, "trailingPE")),
        "forward_pe_ratio": format_number(safe_get(info, "forwardPE")),
        "cash": format_number(safe_get(info, "totalCash")),
        "net_debt": format_number(safe_get(info, "totalDebt") - safe_get(info, "totalCash")) if safe_get(info, "totalDebt") != "-" and safe_get(info, "totalCash") != "-" else "N/A",
        "debt_to_equity": f"{safe_get(info, 'debtToEquity'):.2f}" if safe_get(info, "debtToEquity") != "-" else "-",
        "ebit_to_interest": f"{(safe_get(info, 'ebitda') / safe_get(info, 'interestExpense')):.2f}" if safe_get(info, "ebitda") != "-" and safe_get(info, "interestExpense") != "-" else "N/A",
        "dividendYield": f"{safe_get(info, 'dividendYield')*100:.2f}%" if safe_get(info, "dividendYield") != "-" else "-",
        "payoutRatio": f"{safe_get(info, 'payoutRatio')*100:.2f}%" if safe_get(info, "payoutRatio") != "-" else "-",
        "dividendRate": format_number(safe_get(info, "dividendRate")),
        "revenue_3yr_cagr": "N/A",
        "eps_diluted_3yr_cagr": "N/A"
    }
    return metrics



# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        published_time = pd.to_datetime(item.get("providerPublishTime"), unit='s').strftime('%Y-%m-%d') if item.get("providerPublishTime") else "N/A"
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": published_time
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    try:
        ytd_start_price = history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]
        ytd_return = ((current_price - ytd_start_price) / ytd_start_price) * 100
    except IndexError:
        ytd_return = 0

    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": format_number(current_price),
        "52_week_range": f"{format_number(fifty_two_week_low)} - {format_number(fifty_two_week_high)}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to create an interactive stock chart using Plotly
def create_stock_chart(ticker):
    stock = yf.Ticker(ticker)
    history = stock.history(period="10y")
    if history.empty:
        fig = go.Figure()
        fig.update_layout(
            title=f"No historical data available for {ticker}",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            template="plotly_white"
        )
        return fig

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=history.index,
            y=history['Close'],
            mode='lines',
            name='Close Price',
            line=dict(color='purple'),
            fill='tozeroy',
            fillcolor='rgba(128, 0, 128, 0.2)'  # Semi-transparent purple
        )
    )
    fig.update_layout(
        title=f"Stock Price History for {ticker}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        template="plotly_white",
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey'
        )
    )
    return fig

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Enterprise Value:** {financials.get('enterprise_value')}
- **Shares Outstanding:** {financials.get('shares_outstanding')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Financial Health**
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

**Dividends**
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')}) - {item.get('published_time')}\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get company name from user's conversation dynamically
def get_company_name_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']

    try:
        # Use OpenAI to extract a company name or ticker symbol
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant that extracts stock ticker symbols or company names from user messages."},
                {"role": "user", "content": user_message}
            ],
            max_tokens=20,  # Allow enough tokens to capture a name or ticker
            temperature=0
        )

        # Extract the name or ticker symbol from the response
        extracted_name = response['choices'][0]['message']['content'].strip()

        # Basic validation to check if the extracted name seems like a company (could be improved)
        if re.match(r'^[A-Za-z\s]+$', extracted_name):  # Ensure it's a valid company name (letters and spaces)
            return extracted_name
        else:
            return None
    except Exception as e:
        # Handle any errors from OpenAI request
        print(f"Error during company name extraction: {e}")
        return None

# Function to handle polite fallback when no specific ticker is found
def handle_no_stock_found(company_name=None):
    if company_name:
        return f"""
It seems like you might be asking about the performance of a company such as "{company_name}".
However, I need more specific information to proceed, such as the full name or ticker symbol of the company.
Could you please clarify?
"""
    else:
        return """
It seems like you're asking about the performance of a company, but I couldn't identify a specific company name.
Could you please provide more specific information, such as the full name or ticker symbol?
"""

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    try:
        # Define the system prompt
        system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
        # Prepare the messages
        messages = [{"role": "system", "content": system_prompt}] + conversation

        # Send the conversation to OpenAI
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            functions=functions,
            function_call="auto"
        )

        message = response['choices'][0]['message']

        # Check if the assistant wants to call a function
        if message.get("function_call"):
            function_name = message["function_call"]["name"]
            function_args = json.loads(message["function_call"]["arguments"])

            # Retrieve the ticker if not provided
            ticker = function_args.get("ticker")
            if not ticker:
                # Try extracting a company name from the conversation
                company_name = get_company_name_from_conversation(conversation)
                if not company_name:
                    # Handle case where no company or ticker is found
                    return handle_no_stock_found()

                # Handle case where company name is found but no ticker is provided
                return handle_no_stock_found(company_name)

            # Update the stock_ticker state with the found or suggested ticker
            stock_ticker["value"] = ticker

            # Dynamically call the appropriate function based on the function name
            if function_name == "get_full_stock_report":
                report = get_full_stock_report(ticker)
                formatted_report = format_stock_report(report)
                conversation.append({"role": "function", "name": function_name, "content": formatted_report})

                final_response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=conversation
                )
                assistant_message = final_response['choices'][0]['message']['content']
                conversation.append({"role": "assistant", "content": assistant_message})
                return assistant_message

            # Handle other function calls similarly...

        else:
            # If no function call, return the assistant's message
            assistant_response = message.get("content")
            conversation.append({"role": "assistant", "content": assistant_response})
            return assistant_response
    except Exception as e:
        # Handle any general errors
        print(f"Error in stock chat: {e}")
        return "I'm sorry, something went wrong while processing your request. Could you please try again?"

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# Function to display stock insight
def display_stock_insight(ticker):
    overview = get_company_overview(ticker)
    financials = get_financial_metrics(ticker)
    performance = get_stock_performance(ticker)

    stock_insight_content = f"""
### Company Overview
- **Name:** {overview.get('company_name')}
- **CEO:** {overview.get('ceo')}
- **Website:** [{overview.get('website') or 'N/A'}](https://{overview.get('website')}) if overview.get('website') else 'N/A'
- **Sector:** {overview.get('industry')}
- **Year Founded:** {overview.get('year_founded')}

### Profile
- **Market Cap:** {financials.get('market_cap')}
- **EV:** {financials.get('enterprise_value')}
- **Shares Out:** {financials.get('shares_outstanding')}
- **Revenue:** {financials.get('total_revenue')}
- **Employees:** {financials.get('employees')}

### Margins
- **Gross Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}

### Stock Performance
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}

### Financial Health
- **Cash:** {financials.get('cash')}
- **Net Debt:** {financials.get('net_debt')}
- **Debt/Equity:** {financials.get('debt_to_equity')}
- **EBIT/Interest:** {financials.get('ebit_to_interest')}

### Growth (CAGR)
- **Revenue 3Yr:** {financials.get('revenue_3yr_cagr')}
- **EPS Diluted 3Yr:** {financials.get('eps_diluted_3yr_cagr')}

### Dividends
- **Yield:** {financials.get('dividendYield')}
- **Payout Ratio:** {financials.get('payoutRatio')}
- **DPS:** {financials.get('dividendRate')}

**Description:** {overview.get('description')}
"""
    return stock_insight_content

# NEW CODE: Create Gradio interface with custom layout
with gr.Blocks(css="""
/* Custom CSS for White Chat Background and Enhanced Appearance */

/* Style the entire Gradio container */
.gradio-container {
    background-color: #ffffff; /* White background for the entire application */
}

/* Style individual chat messages */
.chatbot .message {
    background-color: #ffffff; /* White background for messages */
    border-radius: 10px;
    padding: 10px;
    margin-bottom: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}

/* Differentiate user and assistant messages with border accents */
.chatbot .message.user {
    border-left: 4px solid #6c757d; /* Grey accent for user messages */
}

.chatbot .message.assistant {
    border-left: 4px solid #6c757d; /* Grey accent for assistant messages */
}

/* Optional: Style the chat scrollbar */
.chatbot .scroll-container::-webkit-scrollbar {
    width: 8px;
}

.chatbot .scroll-container::-webkit-scrollbar-track {
    background: #f1f1f1;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb {
    background: #888;
    border-radius: 4px;
}

.chatbot .scroll-container::-webkit-scrollbar-thumb:hover {
    background: #555;
}

/* Style the Stock Insight section */
#stock_insight_section {
    background-color: #ffffff; /* White background */
    padding: 15px;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow for depth */
}

/* Animation for the textbox placeholder */
@keyframes placeholderMove {
    0% {
        opacity: 0;
        transform: translateX(0);
    }
    50% {
        opacity: 1;
        transform: translateX(10px);
    }
    100% {
        opacity: 0;
        transform: translateX(-10px);
    }
}

.animated-placeholder::placeholder {
    animation: placeholderMove 4s infinite;
    color: #6c757d; /* Grey color for placeholder text */
}
""") as demo:
    gr.Markdown("<h1><center>Talk 2 Stock</center></h1>")
    with gr.Row():
        with gr.Column(scale=6):  # 60% width
            gr.Markdown("<h2><center>Chat Xchange</center></h2>")
            chatbot = gr.Chatbot()
            state = gr.State([])  # To store the conversation history
            with gr.Row():
                txt = gr.Textbox(
                    show_label=False,
                    placeholder="Hey,Trade.         How is Tesla doing today?",
                    elem_classes="animated-placeholder"
                )
                txt.submit(chat, inputs=[txt, state], outputs=[chatbot, state])
                txt.submit(lambda: "", None, txt)  # Clear the input box after submission
        with gr.Column(scale=4):  # 40% width
            gr.Markdown("<h2><center>Stock Insight</center></h2>")
            with gr.Column(elem_id="stock_insight_section"):
                stock_chart = gr.Plot()
                stock_insight = gr.Markdown()
            # Function to update stock insight
            state.change(
                update_stock_insight,
                inputs=state,
                outputs=[stock_chart, stock_insight]
            )

# Launch the Gradio app
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1123ee42d7d494e408.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
